<a href="https://colab.research.google.com/github/bilgeyucel/multilingual-qa/blob/main/Multilingual_QA_with_Cohere_Haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Should I stay at this place?

Multilingual Generative QA Using Cohere and Haystack

In [1]:
!pip install farm-haystack[inference]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [2]:
from getpass import getpass

COHERE_API_KEY = getpass("Enter Cohere API key:")

Enter Cohere API key:··········


In [ ]:
from haystack.nodes import EmbeddingRetriever, PreProcessor
from haystack.document_stores import InMemoryDocumentStore
from haystack.pipelines import Pipeline
from haystack.schema import Document


document_store = InMemoryDocumentStore(embedding_dim=1024, similarity= "dot_product")
retriever = EmbeddingRetriever(
    embedding_model="embed-multilingual-v2.0",
    document_store=document_store,
    api_key=COHERE_API_KEY
)
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=False,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
)

documents = [Document("O ar condicionado de um dos quartos deu problema, mas levaram um ventilador para ser utilizado. Também por ser em uma área bem movimentada, o barulho da rua pode ser ouvido. Porém, eles deixam protetores auriculares para o uso. Também senti falta de um espelho de corpo inteiro no apartamento. Só havia o do banheiro que mostra apenas a parte superior do corpo."),
             Document("Durchgängig Lärm, weil direkt an der Partymeile; schmutziges Geschirr; unvollständige Küchenausstattung; Abzugshaube über Herd ging für zwei Stunden automatisch an und lies sich nicht abstellen; Reaktionen auf Anfragen entweder gar nicht oder unfreundlich"),
             Document("Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen. Auch das Angebot des Shuttleservices war super und würde ich empfehlen - sehr unkompliziert! Unser Flug hatte Verspätung und der Shuttle hat auf uns gewartet. Die Lage zur Innenstadt ist sehr gut,jedoch ist die Fensterfront direkt zur Club-Straße deshalb war es nachts bis drei/vier Uhr immer recht laut. Die Kaffeemaschine oder auch die Couch hätten sauberer sein können. Ansonsten war das Appartement aber völlig ok."),
             Document("Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bouchons d'oreilles fournis !)"),
             Document("Zapach moczu przy wejściu do budynku, może warto zainstalować tam mocne światło na czujnik ruchu, dla gości to korzystne a dla kogoś kto chciałby zrobić tam coś innego niekorzystne :-). Świetne lokalizacje w centrum niestety są na to narażane."),
             Document("El apartamento estaba genial y muy céntrico, todo a mano. Al lado de la librería Lello y De la Torre de los clérigos. Está situado en una zona de marcha, así que si vais en fin de semana , habrá ruido, aunque a nosotros no nos molestaba para dormir"),
             Document("The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain anything about facilities, shower head was broken, there's no cleaning and everything else one may need is charged."),
             Document("It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I N G** the appartement presents itself as a elegant and as a place to relax, very wrong place to relax - you cannot sleep in this appartement, even the beds are vibrating from the bass of the clubs in the same building - you get ear plugs from the hotel -> now I understand why -> I missed a trip as it was so loud and I could not hear the alarm next day due to the ear plugs.- there is a green light indicating 'emergency exit' just above the bed, which shines very bright at night - during the arrival process, you felt the urge of the agent to leave as soon as possible. - try to go to 'RVA clerigos appartements' -> same price, super quiet, beautiful, city center and very nice staff (not an agency)- you are basically sleeping next to the fridge, which makes a lot of noise, when the compressor is running -> had to switch it off - but then had no cool food and drinks. - the bed was somehow broken down - the wooden part behind the bed was almost falling appart and some hooks were broken before- when the neighbour room is cooking you hear the fan very loud. I initially thought that I somehow activated the kitchen fan"),
             Document("Un peu salé surtout le sol. Manque de service et de souplesse"),
             Document("De comfort zo centraal voor die prijs."),
             Document("Die Lage war sehr Zentral und man konnte alles sehenswertes zu Fuß erreichen. Wer am Wochenende nachts schlafen möchte, sollte diese Unterkunft auf keinen Fall nehmen. Party direkt vor der Tür so das man denkt, man schläft mitten drin. Sehr Sehr laut also und das bis früh 5 Uhr. Ab 7 kommt dann die Straßenreinigung die keineswegs leiser ist."),
             Document("Ótima escolha! Apartamento confortável e limpo! O RoofTop é otimo para beber um vinho! O apartamento é localizado entre duas ruas de movimento noturno. Porem as janelas, blindam 90% do barulho. Não nos incomodou"),
             Document("Nous avons passé un séjour formidable. Merci aux personnes , le bonjours à Ricardo notre taxi man, très sympathique. Je pense refaire un séjour parmi vous, après le confinement, tout était parfait, surtout leur gentillesse, aucune chaude négative. Je n'ai rien à redire de négative, Ils étaient a notre écoute, un gentil message tout les matins, pour nous demander si nous avions besoins de renseignement et savoir si tout allait bien pendant notre séjour."),
             Document("Boa localização. Bom pequeno almoço. A tv não se encontrava funcional."),
             Document("Céntrico. Muy cómodo para moverse y ver Oporto. Edificio con terraza propia en la última planta. Todo reformado y nuevo. Te traen un estupendo desayuno todas las mañanas al apartamento. Solo que se puede escuchar algo de ruido de la calle a primeras horas de la noche. Es un zona de ocio nocturno. Pero respetan los horarios.")
]

indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=preprocessor, name="preprocessor", inputs=["File"])
indexing_pipeline.add_node(component=retriever, name="retriever", inputs=["preprocessor"])
indexing_pipeline.add_node(component=document_store, name="document_store", inputs=['retriever'])
indexing_pipeline.run(documents=documents)

In [5]:
from haystack.nodes import AnswerParser, PromptNode, PromptTemplate

prompt="""
You will be provided with reviews in various languages for an accommodation.
Create a concise and informative answer for a given question based solely on the given reviews.
\nReviews: {join(documents)}
\nQuestion: {query};
\nAnswer:
"""
template = PromptTemplate(
    prompt=prompt,
    output_parser=AnswerParser())

prompt_node = PromptNode(model_name_or_path="command", api_key=COHERE_API_KEY, default_prompt_template=template)

rag_pipeline = Pipeline()
rag_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
rag_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])
results = rag_pipeline.run("How is the shuttle service?", params={"Retriever": {"top_k": 3}})
print(results)

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


{'answers': [<Answer {'answer': ' Very convenient. They waited for us when our flight was delayed, and the service is easy to use.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['239fa8f143ed26d7872f2c3d67396b5f', 'a7745e19284ba8ec4f449b928f77a476', '31e8854798e9800383be4ce5806cb6d3'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate a concise and informative answer for a given question based solely on the given reviews.\n\nReviews: Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen. Auch das Angebot des Shuttleservices war super und würde ich empfehlen - sehr unkompliziert! Unser Flug hatte Verspätung und der Shuttle hat auf uns gewartet. Die Lage zur Innenstadt ist sehr gut,jedoch ist die Fensterfront direkt zur Club-Straße deshalb war es nachts bis drei/vier Uhr immer recht laut. Die Kaffeemasch

In [6]:
results = rag_pipeline.run("How is the wifi?")
print(results)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  6.13it/s]


{'answers': [<Answer {'answer': ' The wifi is an area that needs serious improvement. Multiple reviewers have stated that the wifi did not work. ', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['a7745e19284ba8ec4f449b928f77a476', '31e8854798e9800383be4ce5806cb6d3', '29b9716e0b465d728bc64b7740312b48', '5b171ca2c36c4a922f9cd3a1c460fad', '733a5b6224505eb9359c62160c55a784', '3fbea54c97d2ca8ac0ae98abda19b5e8', '239fa8f143ed26d7872f2c3d67396b5f', 'c97add87d9e959b87d5b3b300e39f98', '116c994d9b74fc37970a243aa896f58', '34a14bdffee8f4ac5e401e12ecb84e2b'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate a concise and informative answer for a given question based solely on the given reviews.\n\nReviews: The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain 

In [7]:
results = rag_pipeline.run("Is this place loud?")
print(results)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


{'answers': [<Answer {'answer': ' Several reviewers mention noise as a recurring problem. In particular, the street noise from the nightlife is a problem, as is the noise from other guests in the accommodation. It seems that noise mitigation efforts (such as soundproof windows) are sometimes effective, however.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['c97add87d9e959b87d5b3b300e39f98', 'a7745e19284ba8ec4f449b928f77a476', '116c994d9b74fc37970a243aa896f58', '5b171ca2c36c4a922f9cd3a1c460fad', '3fbea54c97d2ca8ac0ae98abda19b5e8', '82e5d6de9f77ce3076c3906b2ed2baf2', '883cb92bef88095605d65f8d0f058de2', 'e24fa75d570f990efa91a3616b0604de', '34a14bdffee8f4ac5e401e12ecb84e2b', '31e8854798e9800383be4ce5806cb6d3'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate a concise and informative answer for a given question based solely on the given reviews.\n\n

In [10]:
results = rag_pipeline.run("How can I use the coffee maker?")
print(results)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  6.47it/s]


{'answers': [<Answer {'answer': ' We are unable to find relevant information on using a coffee maker in the provided reviews. ', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['a7745e19284ba8ec4f449b928f77a476', 'c97add87d9e959b87d5b3b300e39f98', '34a14bdffee8f4ac5e401e12ecb84e2b', '733a5b6224505eb9359c62160c55a784', '31e8854798e9800383be4ce5806cb6d3', '5b171ca2c36c4a922f9cd3a1c460fad', '29b9716e0b465d728bc64b7740312b48', '82e5d6de9f77ce3076c3906b2ed2baf2', '116c994d9b74fc37970a243aa896f58', '883cb92bef88095605d65f8d0f058de2'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate a concise and informative answer for a given question based solely on the given reviews.\n\nReviews: The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain anything about faci

In [9]:
results = rag_pipeline.run("What are the attractions around this place?")
print(results)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


{'answers': [<Answer {'answer': ' There are several attractions nearby, including the Oporto City Hall, the São Bento Train Station, and the Museum of Port Wine. Additionally, the apartment is located in the heart of the city, making it easy to explore other attractions, such as the Riverside area, the Clérigos Tower, and the nearby beaches. It is also walking distance from the Aliados Metro Station.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['a7745e19284ba8ec4f449b928f77a476', '3fbea54c97d2ca8ac0ae98abda19b5e8', '116c994d9b74fc37970a243aa896f58', '82e5d6de9f77ce3076c3906b2ed2baf2', 'e7fedbc9f5be5dbecbec03581e60dc1e', '883cb92bef88095605d65f8d0f058de2', '34a14bdffee8f4ac5e401e12ecb84e2b', 'c97add87d9e959b87d5b3b300e39f98', '733a5b6224505eb9359c62160c55a784', '29b9716e0b465d728bc64b7740312b48'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate 

In [12]:
results = rag_pipeline.run("Are there pubs near by?")
print(results)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  6.66it/s]


{'answers': [<Answer {'answer': ' Yes, there are several pubs within a short walking distance from the accommodations.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['883cb92bef88095605d65f8d0f058de2', 'a7745e19284ba8ec4f449b928f77a476', '3fbea54c97d2ca8ac0ae98abda19b5e8', '82e5d6de9f77ce3076c3906b2ed2baf2', '733a5b6224505eb9359c62160c55a784', '34a14bdffee8f4ac5e401e12ecb84e2b', 'e7fedbc9f5be5dbecbec03581e60dc1e', 'fb027142af8ff79e551fef6b2d5225fa', 'c97add87d9e959b87d5b3b300e39f98', 'e24fa75d570f990efa91a3616b0604de'], 'meta': {'prompt': "\nYou will be provided with reviews in various languages for an accommodation.\nCreate a concise and informative answer for a given question based solely on the given reviews.\n\nReviews: Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bouchons d'oreilles fournis !) The keypad with a code is convenient and the location 

In [ ]:
QUESTIONS = [
    "How is the wifi?",
    "How can I use the coffee maker?",
    "Is this place loud?",
    "Are there pubs near by?"
]